# Pulitzer Insights

Q1 - Which newspaper is getting Pulitzer ?

Reading Data from GCP

In [ ]:
import pandas as pd
from google.cloud import storage

import datetime as dt
from datetime import datetime
from pytz import timezone

import uuid

#Reading Google Buckets for files
client = storage.Client()
bucket=client.get_bucket('capstone_project_sr')
blob = storage.Blob('pulitzer.csv',bucket)
with open('pulitzer.csv', 'wb') as file_obj:
    blob.download_to_file(file_obj)
df=pd.read_csv('pulitzer.csv',sep=',',header=0, names=['Newspaper','DailyCirculation_2004',\
                                                       'DailyCirculation_2013',\
                                                       'ChangeInDailyCirculation_2004_2013',\
                                                       'PulitzerPrizeWinnersAndFinalists_1990_2003',\
                                                       'PulitzerPrizeWinnersAndFinalists_2004_2014',\
                                                       'PulitzerPrizeWinnersAndFinalists_1990_2014'])

#Adding audit details
tz = timezone('EST')
datetime.now(tz) 
df['Entrydate'] = dt.datetime.now()

df.insert(0,'Id',uuid.uuid4())
df.Id= df.Id.apply(lambda x: uuid.uuid4())

Insert Data into Cassandra Cluster so that data can be audited and preserved.

In [ ]:
import itertools
from multiprocessing import Pool
import sys
import time
from cassandra.cluster import Cluster
from cassandra.concurrent import execute_concurrent_with_args
from cassandra.query import tuple_factory
from cassandra.auth import PlainTextAuthProvider

def _insertData(params):
    #cluster = Cluster()
    cluster = Cluster(contact_points=['10.128.0.4'], auth_provider = PlainTextAuthProvider(username='cuser', password='gYZ9u58vOMU='))
    session = cluster.connect()
    session.set_keyspace('somatic')
    session.row_factory = tuple_factory
    #prepared=session.prepare("INSERT INTO mytable (a, b) VALUES (?, ?)")
    prepared=session.prepare("INSERT INTO capstone.pulitzer \
                            (id,Newspaper,DailyCirculation_2004,DailyCirculation_2013, \
                             ChangeInDailyCirculation_2004_2013,PulitzerPrizeWinnersAndFinalists_1990_2003, \
                             PulitzerPrizeWinnersAndFinalists_2004_2014, \
                             PulitzerPrizeWinnersAndFinalists_1990_2014,Entrydate) \
                             VALUES (?,?,?,?,?,?,?,?,?)")
    
    execute_concurrent_with_args(session, prepared, params, concurrency=50)
    return None

def multiprocess(params):
    pool = Pool(processes=4)
    results = [pool.map(_insertData, (params[n:n+100],)) for n in range(0, len(params),100)]
    return results
    

if __name__ == "__main__":
    parameters=[]
    for index, row in enumerate(df.values):        
        (a,b,c,d,e,f,g,h,i) = row
        row1=(a,str(b),str(c),str(d),str(e),str(f),str(g),str(h),i)
        parameters.append(row1)           
    #parameters = [(x,x) for x in range(10000)]
    a = multiprocess(parameters)